# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [177]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
# Import API key
from api_keys import geoapify_key
import requests
from requests.structures import CaseInsensitiveDict
import folium
from folium import plugins
import json
import warnings
warnings.filterwarnings('ignore')
# import cartopy.crs as ccrs
# import geoviews as gv # noqa


# url = "https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20Westminster%20W1H%201LJ%2C%20United%20Kingdom&apiKey=" + geoapify_key

# headers = CaseInsensitiveDict()
# headers["Accept"] = "application/json"

# resp = requests.get(url, headers=headers)

# print(resp.status_code)


In [178]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,airai,-8.9266,125.4092,18.52,89,100,1.10,TL,1679380430
1,1,cockburn town,21.4612,-71.1419,25.27,80,2,3.57,TC,1679380405
2,2,hambantota,6.1241,81.1185,32.19,66,40,2.57,LK,1679380431
3,3,rikitea,-23.1203,-134.9692,26.55,77,93,6.16,PF,1679380432
4,4,mataura,-46.1927,168.8643,8.16,73,97,2.31,NZ,1679380432
...,...,...,...,...,...,...,...,...,...,...
560,560,maarianhamina,60.0973,19.9348,-0.66,100,100,2.06,AX,1679380784
561,561,nador,35.1740,-2.9287,13.14,82,20,2.06,MA,1679380785
562,562,dalianwan,39.0286,121.6950,12.02,58,8,6.00,CN,1679380786
563,563,veraval,20.9000,70.3667,27.44,56,48,4.55,IN,1679380786


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [179]:
# %%capture --no-display
# Configure the map plot

""" 
I could not install the library geoViews, I broke my anaconda environment trying to install this library in an unconventional way by manually downloading dependencies
see below and also downloading the library and installing locally without luck, so I gave folium a try. 

Nothing worked for geoViews. Spent too much time trying to make it work unsuccessfully.
Tried with python 3.7.9, 3.9.13 in a clean installatioin in my OS and in the anaconda environments.  

So instead of adding the humidity as the size of the marker, I added a tooltip with that shows the humidity, please see the HTML file and click on the cities to see it working. 
Other than that I feel I completed all that was requested in the challenge. Thank you! 

Cartopy-0.20.2-cp39-cp39-win_amd64.whl
Shapely-1.8.2-cp39-cp39-win_amd64.whl
geoviews-1.9.6-py2.py3-none-any.whl

In anaconda I get "Invoke-Expression : Cannot bind argument to parameter 'Command' because it is an empty string."
could not use anaconda, had to uninstall and reinstall it. 

 "conda install -c pyviz hvplot geoviews",  , xlim=(-180, -30), ylim=(0,75) ,plot_height=1000, plot_width=1000

"""
m = folium.Map() # I had issues using hvplot due to dependancies and left this folium code that perhaps I was not going to be able to make it work. 
for i in range (0, len(city_data_df)):
    folium.Marker([city_data_df.iloc[i]['Lat'], \
                city_data_df.iloc[i]['Lng']], popup=city_data_df.iloc[i]['City'], \
                tooltip="HUMIDITY - " + str(city_data_df.iloc[i]['Humidity']) ).add_to(m)
m.save("output_data/map_created_with_folium.html")
print("To interact with the Folium MAP go to output_data/map.html" )

m2 = city_data_df.hvplot.points('Lng','Lat', geo=True, color='blue', tiles='OSM', hover_cols='City', width=1200)
hvplot.save(m2, "output_data/map_created_with_hvplot.html")
# Display the map
m2 

To interact with the Folium MAP go to output_data/map.html


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [189]:
# Narrow down cities that fit criteria and drop any results with null values
#Units are metric, so teperature is in celcius, wind speed in kilometers
# Cloud amount is reported in oktas or eighths with the additional convention that:
# 0 oktas represents the complete absence of cloud
# 1 okta represents a cloud amount of 1 eighth or less, but not zero
# 7 oktas represents a cloud amount of 7 eighths or more, but not full cloud cover
# 8 oktas represents full cloud cover with no breaks
# 9 oktas represents sky obscured by fog or other meteorological phenomena
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27.78) & (city_data_df["Max Temp"] > 20.56) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] < 10)] 
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
# Display sample data
print("Total Ideal Weather Cities " + str(len(ideal_weather_df)))
ideal_weather_df

Total Ideal Weather Cities 32


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,cockburn town,21.4612,-71.1419,25.27,80,2,3.57,TC,1679380405
51,51,port alfred,-33.5906,26.8910,21.21,81,1,1.79,ZA,1679380463
76,76,champerico,14.3000,-91.9167,26.81,77,6,1.50,GT,1679380478
81,81,arraial do cabo,-22.9661,-42.0278,24.92,88,0,4.63,BR,1679380296
136,136,adre,13.4667,22.2000,27.25,13,8,6.81,TD,1679380515
142,142,siguiri,11.6667,-9.5000,24.52,17,6,0.69,GN,1679380519
158,158,khash,28.2211,61.2158,20.66,23,0,7.09,IR,1679380531
161,161,morondava,-20.2833,44.2833,27.56,71,0,0.66,MG,1679380533
173,173,iquitos,-3.7481,-73.2472,24.02,94,0,0.00,PE,1679380541
193,193,turbat,26.0023,63.0440,27.55,37,0,2.74,PK,1679380553


### Step 3: Create a new DataFrame called `hotel_df`.

In [190]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = 0
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [191]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (788381735.py, line 2)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE